In [1]:
import csv
import urllib
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
soup = BeautifulSoup(urllib.request.urlopen('https://www.kaggle.com/c/outbrain-click-prediction/data'), "lxml")
headers = [header.text for header in soup.findAll('th')]
rows = []
for row in soup.find(id="data-files"):
    rows.append([val.attrs['href'] for val in row.find_all_next('a', limit=11)])

links = ['links']+(rows[0][0:11])

with open('kaggle_outbrain_data_links.csv', 'w') as f:
    writer = csv.writer(f, lineterminator='\n')
    for val in links:
        writer.writerow([val])  


In [3]:
test = pd.read_csv("kaggle_outbrain_data_links.csv")

In [4]:
test['filename'] = test['links'].str.replace("/c/outbrain-click-prediction/download/","").str.replace(".zip", "")
test['links'] = test['links'].str.replace("/c/", "https://www.kaggle.com/c/")

In [5]:
test.to_csv("kaggle_outbrain_data_links.csv")
test.head() 

,links,filename
0,https://www.kaggle.com/c/outbrain-click-predic...,documents_categories.csv
1,https://www.kaggle.com/c/outbrain-click-predic...,clicks_test.csv
2,https://www.kaggle.com/c/outbrain-click-predic...,documents_meta.csv
3,https://www.kaggle.com/c/outbrain-click-predic...,documents_entities.csv
4,https://www.kaggle.com/c/outbrain-click-predic...,promoted_content.csv


In [6]:
import wget
import os
import zipfile
import pandas as pd

def download_if_needed(URL, filename):
    """
    Download from URL to filename unless filename already exists
    """
    if os.path.exists(filename):
        print(filename, 'already exists')
        return
    else:
        print('downloading', filename)
        wget.download(URL)

def get_documents_categories():
    """
    Download pronto data, unless already downloaded
    """
    links_data = pd.read_csv("kaggle_outbrain_data_links.csv")
    download_if_needed(links_data['links'][0],
                       links_data['filename'][0])
    zf = zipfile.ZipFile('documents_categories.csv.zip')
    file_handle = zf.open('documents_categories.csv')
    return pd.read_csv(file_handle)


In [7]:
#wget -x --load-cookies cookies.txt -P . -nH --cut-dirs=5 https://www.kaggle.com/c/outbrain-click-prediction/download/documents_categories.csv.zip

In [8]:
import http.cookiejar, urllib.request
cj = http.cookiejar.CookieJar()
opener = urllib.request.build_opener(urllib.request.HTTPCookieProcessor(cj))
r = opener.open("http://example.com/")
r.code

200

response from 1st place finisher about solution from event-recommendation-engine challenge: https://www.kaggle.com/c/event-recommendation-engine-challenge/forums/t/3894/solutions

this competition was basically all about feature creation. So here is a list of my some of my features:

* There were 3 different dates ( the start time of the event, the jointed_at date, the timestamp). I added all time differences between each dates. They were all important.
* Additionally I extracted the year, month, day, hour and weekday of each date, but of these features had only a small impact.
* I clustered all events using only the c_1, c_2, ..., c_n-features with kmeans (about 30 categories).
* I counted how many events each user had in the train/test file (very important feature).
* I defined similar users as users, which have about the same age, same timezone and same gender and counted how many of them had a yes/maybe/no/invited for the given event. However, these features had no big impact.
* Country was important, too. I only used the 10 most common countries and put all others into the category "Other". I did the same for locale and city.
* The "usual" stuff: Count of yes/maybe/no/invited of all friends and all other users. Lat and lng were very important as well
* In the end I had about 60-70 features, which were all more or less important.

With all these features I tried to predict the "interested"-column and for this I used the probability-output of a gbm. My aim was to minimize the LogLoss-error between my prediction and the actual solution. I never used the MAP-metric, because the lower the LogLoss-error the higher the MAP.

The create a submission I ordered all events using my probability-prediction.